# Computing the formation energy using data from the JANAF tables

The Gibb's free energy is:

$$G = H-TS$$

This gives us the form:
$$ G = aT + b$$ 
where 
$$ a = -S, \quad b = H $$

Now we can compute a least-squares fit to the standard Gibbs free energy of formation ($\Delta _f G^o$) data in the NIST JANAF tables (units kJ/mol), where the physical meaning of the fit parameters will be as follows:

$$a = - \Delta _f S^o, \quad b = \Delta _f H^o$$
with units kJ/mol/K and kJ/mol, respectively

Using the fit parameters $a$ and $b$ we can directly compute the standard Gibbs free energy of formation at temperature $T$. We can use the computed $\Delta _f G^o$ to calculate the equilibrium constant $K$ since they are related according to: 

$$\ln{K_{eq}} = -\frac{G}{RT}$$

Logarithm base change:
$$\log_{10}{K_{eq}} = \ln{K_{eq}} / \ln{10}$$

By substitution, where $\log_{10}{K_{eq}}$ is the same as $\log {\rm K_f}$ in the JANAF tables. To test that our fitting returns results that match the JANAF Tables within fitting precision, we can compute the equilibrium constant as follows using our linear fit for $G$ above:

$$\log K_f = - \frac{\Delta _f G^o}{\ln(10)RT}$$

The equilibrium constant is used in the Law of Mass Action

In [66]:
import pandas as pd
import numpy as np

from atmodeller import TEMPERATURE_JANAF_HIGH, TEMPERATURE_JANAF_LOW, GAS_CONSTANT, logger

### Function that performs a linear fit to the Gibbs free energy data over a particular temperature range:

In [52]:
def least_squares_fit_to_janaf_G(molecule:str):
    """Fit the JANAF Gibbs free energy (G).
    
    Args:
        molecule: The molecule.

    Returns:
        The least-squares fit coefficients.
    """
    datafile: str = f"data/tables/{molecule}.dat"
    df:pd.DataFrame = pd.read_csv(datafile, sep='\t')
    # Filter the data between the relevant temperature range.
    df = df.loc[(df['T(K)'] <= TEMPERATURE_JANAF_HIGH) & (df['T(K)'] >= TEMPERATURE_JANAF_LOW)]
    # Least squares fitting.
    temperature: np.ndarray = df['T(K)'].to_numpy()
    Gf: np.ndarray = df['delta-f G'].to_numpy()
    design_matrix: np.ndarray = temperature[:, np.newaxis]**[1, 0]
    #print(design_matrix)
    solution, _, _, _ = np.linalg.lstsq(design_matrix, Gf, rcond=None)

    return solution

### Compute linear fit parameters to G for a set of molecular species 

In [67]:
for molecule in ("CO", "CO2", "CH4", "H2O", "SO", "SO2", "H2S", "S", "Cl", "F", "NH3"):
    a, b = least_squares_fit_to_janaf_G(molecule)
    out: str = f"{molecule}: a = {a}, b = {b}"
    logger.info(out)

11:47:01 - atmodeller           - INFO      - CO: a = -0.08269582352941174, b = -120.3571470588237
11:47:01 - atmodeller           - INFO      - CO2: a = 0.0005482647058825124, b = -397.33434558823564
11:47:01 - atmodeller           - INFO      - CH4: a = 0.11162272058823527, b = -92.46793382352928
11:47:01 - atmodeller           - INFO      - H2O: a = 0.05817258823529415, b = -251.80119852941178
11:47:01 - atmodeller           - INFO      - SO: a = -0.004834249999999983, b = -59.61862500000004
11:47:01 - atmodeller           - INFO      - SO2: a = 0.0725481764705883, b = -361.0377720588236
11:47:01 - atmodeller           - INFO      - H2S: a = 0.04955845588235296, b = -90.57971323529412
11:47:01 - atmodeller           - INFO      - S: a = -0.06120597058823535, b = 217.87218382352947
11:47:01 - atmodeller           - INFO      - Cl: a = -0.06116926470588236, b = 127.34372058823527
11:47:01 - atmodeller           - INFO      - F: a = -0.06466483823529413, b = 84.63857352941173
11:47:01 

### Testing the fit:

In [74]:
#First, pick a species you'd like to test and run the least-squares fit:
species_test = "NH3"
a_test, b_test = least_squares_fit_to_janaf_G(species_test)
print(a_test, b_test)

0.11667370588235293 -54.026338235293984


In [75]:
#Now let's see how close the fitted G values are to the JANAF values:

test_datafile: str = f"data/tables/{species_test}.dat"
df_test:pd.DataFrame = pd.read_csv(test_datafile, sep='\t')
df_test = df_test.loc[(df_test['T(K)'] <= TEMPERATURE_JANAF_HIGH) & (df_test['T(K)'] >= TEMPERATURE_JANAF_LOW)]

print('Species is:', species_test)
Gf_calc_test = ((a_test*df_test['T(K)']) + b_test)
print('Fitted Gf:')
print(Gf_calc_test)
print('')
print('Difference between fitted G and JANAF values:')
print(Gf_calc_test - df_test['delta-f G'])


Species is: NH3
Fitted Gf:
15    120.984221
16    132.651591
17    144.318962
18    155.986332
19    167.653703
20    179.321074
21    190.988444
22    202.655815
23    214.323185
24    225.990556
25    237.657926
26    249.325297
27    260.992668
28    272.660038
29    284.327409
30    295.994779
Name: T(K), dtype: float64

Difference between fitted G and JANAF values:
15    0.288221
16    0.182591
17    0.084962
18    0.000332
19   -0.071297
20   -0.125926
21   -0.163556
22   -0.184185
23   -0.185815
24   -0.169444
25   -0.134074
26   -0.080703
27   -0.010332
28    0.079038
29    0.184409
30    0.305779
dtype: float64

Calculated logKf from Gf:
15   -4.212963
16   -4.330546
17   -4.434295
18   -4.526517
19   -4.609031
20   -4.683294
21   -4.750485
22   -4.811567
23   -4.867337
24   -4.918460
25   -4.965494
26   -5.008909
27   -5.049108
28   -5.086436
29   -5.121190
30   -5.153626
Name: T(K), dtype: float64


In [77]:
#Finally, let's see how well our fitted Gf values reproduce logKf:

logKf_calc_test = -Gf_calc_test/(np.log(10)*(GAS_CONSTANT/1000.0)*df_test['T(K)'])
print('')
print('Calculated logKf from Gf:')
print(logKf_calc_test)
print('')
print('Difference between calculate logKf and JANAF logKf:')
print(logKf_calc_test - df_test['log Kf'])


Calculated logKf from Gf:
15   -4.212963
16   -4.330546
17   -4.434295
18   -4.526517
19   -4.609031
20   -4.683294
21   -4.750485
22   -4.811567
23   -4.867337
24   -4.918460
25   -4.965494
26   -5.008909
27   -5.049108
28   -5.086436
29   -5.121190
30   -5.153626
Name: T(K), dtype: float64

Difference between calculate logKf and JANAF logKf:
15   -0.009963
16   -0.005546
17   -0.002295
18    0.000483
19    0.001969
20    0.003706
21    0.004515
22    0.004433
23    0.004663
24    0.003540
25    0.002506
26    0.002091
27   -0.000108
28   -0.001436
29   -0.003190
30   -0.005626
dtype: float64
